In [ ]:
%matplotlib inline 

from dolfin import *
# import matplotlib.pyplot as plt
import matplotlib.pyplot as plt


import numpy as np

plt.close()
T_final = 500.0 # final time
dt = 2.0
num_steps =int(T_final/dt)     # number of time steps

# Create classes for defining parts of the boundaries and the interior
# of the domain
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0.0)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 1.0)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0)

class Top_Left(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[1], 1.0) and between(x[0], (0, 0.49)))
    
class Top_Center(SubDomain):
    def inside(self, x, on_boundary):
         return (near(x[1], 1.0) and between(x[0], (0.49, 0.51)))
    
class Top_Right(SubDomain):
    def inside(self, x, on_boundary):
         return (near(x[1], 1.0) and between(x[0], (0.51, 1.0)))

# Initialize sub-domain instances
left = Left()
top_l = Top_Left()
top_c = Top_Center()
top_r = Top_Right()
right = Right()
bottom = Bottom()

# Define mesh
mesh = UnitSquareMesh(100, 100)

# Initialize mesh function for interior domains
domains = MeshFunction("size_t", mesh, mesh.topology().dim())
domains.set_all(0)

# Initialize mesh function for boundary domains
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
boundaries.set_all(0)
left.mark(boundaries, 1)
right.mark(boundaries, 3)
bottom.mark(boundaries, 2)
top_l.mark(boundaries, 4)
top_c.mark(boundaries, 5)
top_r.mark(boundaries, 6)

rho_0 = 1650 #original 800
c_0 = 830 #original 830
#convection coefficient. 
h_conv = 11.0
# kappa not in use with steady state solutions 
kappa_0 = 2.03
T_0 = 313.0

# Define function space and basis functions
V = FunctionSpace(mesh, "CG", 2)
u = TrialFunction(V)
v = TestFunction(V)

u_init = Constant('313.0')
u_n = interpolate(u_init, V)

g_0 = Constant('230.0')
g_1 = Constant('0')

# Define new measures associated with the interior domains and
# exterior boundaries
dx = Measure('dx', domain=mesh, subdomain_data=domains)
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

a = rho_0*c_0*u*v*dx + kappa_0*dt*dot(grad(u), grad(v))*dx(0) + kappa_0*h_conv*(u)*v*dt*(ds(1) + ds(2) + ds(3))
L = rho_0*c_0*u_n*v*dx(0)  + kappa_0*g_0*v*dt*(ds(5)+ds(6)+ds(4)) + kappa_0*h_conv*dt*(T_0)*v*(ds(1) + ds(2) + ds(3))

# Time-stepping
u = Function(V)
t = 0

print('Final time: ' + str(T_final))

for n in range(num_steps):

    # Update current time
    t += dt
    
    # Compute solution
    solve(a == L, u)

    # Update previous solution
    u_n.assign(u)
    
    #print('Current time ' + str(t))
#Solve Problem
# u = Function(V)
# solve(a == L, u, bcs)

# Plot solution
plt.figure()
p = plot(u, title="After 500 seconds; Insulated 123 Uniform Flux 456", window_width = 2000)

plt.colorbar(p)
plt.interactive(True)

plt.show()